**This personal project presents an interactive data visualization of National Science Foundation (NSF) grants from fiscal years 2021 to 2025, considering the impact of recent cancellations.**

Using Altair and Streamlit, I developed a set of visualizations following the "Overview first, zoom and filter, then details on demand" methodology. The analysis examines grant distribution across U.S. states and NSF directorates (more than 57,000 total grants), temporal evolution of funding amounts, and the impact of nearly 2,000 grant cancellations during the Trump administration in 2025.

In [1]:
# Required for Collab
!pip install squarify
!pip install "vl-convert-python>=1.0.0"
!pip install vegafusion

In [2]:
# All required libraries
import pandas as pd
import numpy as np
import altair as alt
import squarify
import math
from vega_datasets import data as vega_data

alt.data_transformers.enable('vegafusion') # handling large datasets

DataTransformerRegistry.enable('vegafusion')

### Loading source `NSF Award Search`

In [3]:
df_awards = pd.read_csv('clean_data/nsf_awards_full.csv')

# Missing values
print("MISSING VALUES:")
print(df_awards.isnull().sum())

# Grants per year
print("\n")
print("GRANTS PER YEAR:")
print(df_awards['Year'].value_counts().sort_index())


MISSING VALUES:
AwardID            0
Directorate        0
DirectorateAbbr    0
Division           0
DivisionAbbr       0
StateCode          0
StateName          0
Year               0
EstimatedBudget    0
dtype: int64


GRANTS PER YEAR:
Year
2021    12078
2022    11817
2023    11966
2024    11643
2025     9204
Name: count, dtype: int64


### Loading source `US_States.csv`

In [4]:
df_states = pd.read_csv('clean_data/us_states.csv')

# Missing values
print("MISSING VALUES:")
print(df_states.isnull().sum())

# Dictionary Mappings for fast lookups (Q1-Q5)
state_fips_map = dict(zip(df_states['StateCode'], df_states['Id']))
state_party_2020_map = dict(zip(df_states['StateCode'], df_states['Party2020']))
state_party_2025_map = dict(zip(df_states['StateCode'], df_states['Party2025']))
state_lat_map = dict(zip(df_states['StateCode'], df_states['Latitude']))
state_lon_map = dict(zip(df_states['StateCode'], df_states['Longitude']))

MISSING VALUES:
StateName    0
StateCode    0
Party2020    0
Party2025    0
Latitude     0
Longitude    0
Id           0
dtype: int64


### Loading source `nsf_cancellations.csv`

In [5]:
df_cancellations = pd.read_csv('clean_data/nsf_cancellations.csv')

# Missing values
print("MISSING VALUES:")
print(df_cancellations.isnull().sum())

# Check cancelled grants already in the main dataset
cancelled_ids = set(df_cancellations['AwardID'].astype(str))
main_ids = set(df_awards['AwardID'].astype(str))
overlap = cancelled_ids & main_ids
missing = cancelled_ids - main_ids

print(f"\nCancelled grants in nsf_cancellations.csv: {len(cancelled_ids)}")
print(f"Already in nsf_awards_full.csv: {len(overlap)} ({100*len(overlap)/len(cancelled_ids):.1f}%)")
print(f"Missing from nsf_awards_full.csv: {len(missing)} ({100*len(missing)/len(cancelled_ids):.1f}%)")

state_name_map = dict(zip(df_states['StateCode'], df_states['StateName']))

# Filter missing cancellations
missing_cancellations = df_cancellations[
    df_cancellations['AwardID'].astype(str).isin(missing)
].copy()

missing_cancellations['StateName'] = missing_cancellations['StateCode'].map(state_name_map)
missing_cancellations['Year'] = missing_cancellations['Year'].astype(int)

df_cancellations['StateName'] = df_cancellations['StateCode'].map(state_name_map)

MISSING VALUES:
AwardID            0
Status             0
Year               0
StateCode          0
EstimatedBudget    0
Directorate        0
DirectorateAbbr    0
Division           0
DivisionAbbr       0
dtype: int64

Cancelled grants in nsf_cancellations.csv: 1816
Already in nsf_awards_full.csv: 1807 (99.5%)
Missing from nsf_awards_full.csv: 9 (0.5%)


### Dataframe including all grants, cancelled and non-cancelled (2021-2025)

In [6]:
# Combined df_awards (main + missing cancellations)
df_complete = pd.concat([
    df_awards[['AwardID', 'StateCode', 'StateName', 'Year']],
    missing_cancellations[['AwardID', 'StateCode', 'StateName', 'Year']]
], ignore_index=True)

# Add Party
df_complete['Party'] = df_complete.apply(
    lambda row: state_party_2020_map.get(row['StateCode'], 'Unknown') if row['Year'] <= 2024
    else state_party_2025_map.get(row['StateCode'], 'Unknown'), axis=1
)

print("MISSING VALUES:")
print(df_complete.isnull().sum())
print(f"\nAdded {len(missing_cancellations)} cancelled grants")

MISSING VALUES:
AwardID      0
StateCode    0
StateName    0
Year         0
Party        0
dtype: int64

Added 9 cancelled grants


### Map topology and political party ruling

In [7]:
# Grant Share (%)
df_state_grants = df_complete.groupby(['StateCode', 'StateName', 'Year']).size().reset_index(name='GrantCount')
year_totals = df_state_grants.groupby('Year')['GrantCount'].sum().to_dict()
df_state_grants['GrantRate'] = df_state_grants.apply(
    lambda row: (row['GrantCount'] / year_totals[row['Year']]) * 100, axis=1
)

# Map topology data
df_state_grants['id'] = df_state_grants['StateCode'].map(state_fips_map)
df_state_grants['latitude'] = df_state_grants['StateCode'].map(state_lat_map)
df_state_grants['longitude'] = df_state_grants['StateCode'].map(state_lon_map)

# Political party based on the year
df_state_grants['Party'] = df_state_grants.apply(
    lambda row: state_party_2020_map[row['StateCode']] if row['Year'] <= 2024
    else state_party_2025_map[row['StateCode']], axis=1
)

print("MISSING VALUES:")
print(df_state_grants.isnull().sum())

MISSING VALUES:
StateCode     0
StateName     0
Year          0
GrantCount    0
GrantRate     0
id            0
latitude      0
longitude     0
Party         0
dtype: int64


<br></br>

## **How are the grants distributed by states every year?**

In [8]:
# All Years Average
q1_all_years = df_state_grants.groupby(['StateCode', 'StateName', 'id']).agg({
    'GrantCount': 'sum',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()

total_all_grants = q1_all_years['GrantCount'].sum()
q1_all_years['GrantRate'] = (q1_all_years['GrantCount'] / total_all_grants) * 100

YEAR_ALL_INDICATOR = 0 # Used to represent the average of All Years
NUM_YEARS = 5

q1_all_years['Year'] = YEAR_ALL_INDICATOR
q1_all_years['Party'] = q1_all_years['StateCode'].map(state_party_2020_map)

q1_combined = pd.concat([df_state_grants, q1_all_years], ignore_index=True)

year_options = [YEAR_ALL_INDICATOR, 2021, 2022, 2023, 2024, 2025]
year_labels = ['All', '2021', '2022', '2023', '2024', '2025']
year_input = alt.binding_select(options=year_options, labels=year_labels, name='Year: ')
year_param = alt.param(name='year_select', value=YEAR_ALL_INDICATOR, bind=year_input)

party_input = alt.binding_select(options=['All', 'Democrat', 'Republican'], name='Party: ')
party_param = alt.param(name='party_filter', value='All', bind=party_input)

party_color_scale = alt.Scale(domain=['Democrat', 'Republican'], range=['#4C78A8', '#E45756'])

max_rate = q1_combined['GrantRate'].max()
size_scale = alt.Scale(domain=[0, max_rate], range=[20, 1500])

us_states_geo = alt.topo_feature(vega_data.us_10m.url, "states")

background = alt.Chart(us_states_geo).mark_geoshape(
    fill='#e8e8e8', stroke='white', strokeWidth=0.5
).project('albersUsa')

In [9]:
# CONSTANTS TO ADJUST THE CHART

# Dimensions
MAP_WIDTH = 550
MAP_HEIGHT = 400
BAR_WIDTH = 300
BAR_HEIGHT = 400
LEGEND_WIDTH = 350
LEGEND_SPACER_WIDTH = (MAP_WIDTH - LEGEND_WIDTH) / 2
BAR_CHART_TOP_PADDING_HEIGHT = 36
LEGEND_TEXT_HEIGHT = 20
LEGEND_BAR_HEIGHT = 15

# Colors
COLOR_DEMOCRAT = "#377eb8"
COLOR_REPUBLICAN = "#e41a1c"
COLOR_DEMOCRAT_DARK = "#12129E"
COLOR_REPUBLICAN_DARK = "#8B0000"
COLOR_STROKE_BLACK = "black"
COLOR_STROKE_WHITE = "white"
COLOR_BACKGROUND_MAP = "#e0e0e0"
COLOR_SCHEME_RATE = "teals"

# Styling
STROKE_WIDTH_THIN = 0.5
STROKE_WIDTH_THICK = 4
STROKE_WIDTH_SYMBOL = 5
OPACITY_ACTIVE = 1.0
OPACITY_INACTIVE = 0.0
OPACITY_CIRCLE = 0.9
OPACITY_RULE_OUTLINE = 0.8
CIRCLE_SIZE = 50

# Offsets for legends
PARTY_LEGEND_OFFSET_Y = -20
MEAN_LEGEND_OFFSET = -45

# Interaction parameters
TOP_N_MIN = 5
TOP_N_MAX = 20
TOP_N_STEP = 1
DEFAULT_TOP_N = 10

max_rate = q1_combined['GrantRate'].max()

color_definition = alt.Scale(scheme=COLOR_SCHEME_RATE, domain=[0, max_rate])

party_color_scale = alt.Scale(
    domain=["Democrat", "Republican"],
    range=[COLOR_DEMOCRAT, COLOR_REPUBLICAN]
)

party_legend_colors = alt.Scale(
    domain=["Democrat", "Republican"],
    range=[COLOR_DEMOCRAT_DARK, COLOR_REPUBLICAN_DARK]
)

# INTERACTION PARAMETERS

year_input = alt.binding_select(
    options=[YEAR_ALL_INDICATOR, 2021, 2022, 2023, 2024, 2025],
    labels=['All', '2021', '2022', '2023', '2024', '2025'],
    name='Fiscal Year: '
)
year_param = alt.param(name='year_select', value=YEAR_ALL_INDICATOR, bind=year_input)

topn_slider = alt.binding_range(
    min=TOP_N_MIN, max=TOP_N_MAX, step=TOP_N_STEP, name='Top N States: '
)
topn_param = alt.param(name='topn', value=DEFAULT_TOP_N, bind=topn_slider)

party_input = alt.binding_select(
    options=['All', 'Republican', 'Democrat'],
    name='Party Filter: '
)
party_param = alt.param(name='party_filter', value='All', bind=party_input)

# BAR CHART

base_bars = alt.Chart(q1_combined).transform_filter(
    alt.datum.Year == year_param
).transform_filter(
    "(party_filter == 'All') || (datum.Party == party_filter)"
).transform_window(
    rank='rank(GrantRate)',
    sort=[alt.SortField('GrantRate', order='descending')]
).transform_filter(
    alt.datum.rank <= topn_param
)

y_axis_config = alt.Y(
    'StateName:N',
    title=None,
    sort=alt.EncodingSortField(field="rank", op="min", order="ascending")
)

bars = base_bars.mark_bar(
    stroke=COLOR_STROKE_BLACK,
    strokeWidth=STROKE_WIDTH_THIN
).encode(
    x=alt.X('GrantRate:Q', title='Grant Share (%)', scale=alt.Scale(domain=[0, max_rate])),
    y=y_axis_config,
    color=alt.Color('Party:N', scale=party_color_scale, legend=None),
    tooltip=[
        'StateName:N',
        'Party:N',
        alt.Tooltip('GrantRate:Q', format='.2f', title='GrantRate (%)'),
        alt.Tooltip('GrantCount:Q', format=',')
    ]
).properties(width=BAR_WIDTH, height=BAR_HEIGHT)

rule_base = alt.Chart(q1_combined).transform_filter( # Mean Line calc
    alt.datum.Year == year_param
).transform_filter(
    "(party_filter == 'All') || (datum.Party == party_filter)"
).transform_aggregate(
    GrantRate='mean(GrantRate)', groupby=['Party']
)

rule_outline = rule_base.mark_rule(
    strokeWidth=STROKE_WIDTH_THICK,
    opacity=OPACITY_RULE_OUTLINE
).encode(
    x='GrantRate:Q',
    stroke=alt.value(COLOR_STROKE_BLACK),
    tooltip=[
        alt.Tooltip('Party:N'),
        alt.Tooltip('GrantRate:Q', format='.2f', title='Mean (%)')
    ]
)

rule_color = rule_base.mark_rule(strokeWidth=STROKE_WIDTH_SYMBOL).encode(
    x='GrantRate:Q',
    stroke=alt.Stroke('Party:N', scale=party_legend_colors,
        legend=alt.Legend(
            title='Mean by Party',
            orient='top',
            direction='vertical',
            offset=MEAN_LEGEND_OFFSET,
            symbolStrokeWidth=STROKE_WIDTH_SYMBOL
        )
    ),
    tooltip=[
        alt.Tooltip('Party:N'),
        alt.Tooltip('GrantRate:Q', format='.2f', title='Mean (%)')
    ]
)

bar_chart_base = alt.layer(bars, rule_outline, rule_color).resolve_scale(color='independent')

# Invisible chart to make the bar chart aligned with the map
bar_top_spacer = alt.Chart(pd.DataFrame({'x': [0]})).mark_point(opacity=0).encode().properties(
    width=BAR_WIDTH,
    height=BAR_CHART_TOP_PADDING_HEIGHT
)
bar_chart = alt.vconcat(bar_top_spacer, bar_chart_base, spacing=0)

# CHOROPLETH MAP

background = alt.Chart(us_states_geo).mark_geoshape(
    fill=COLOR_BACKGROUND_MAP,
    stroke=COLOR_STROKE_WHITE,
    strokeWidth=STROKE_WIDTH_THIN
).project('albersUsa')

layers = []
year_list = [YEAR_ALL_INDICATOR, 2021, 2022, 2023, 2024, 2025]

for i, year in enumerate(year_list):
    yd = q1_combined[q1_combined['Year'] == year].copy()

    # Chloropleth
    map_shape = alt.Chart(us_states_geo).mark_geoshape(
        stroke=COLOR_STROKE_WHITE,
        strokeWidth=STROKE_WIDTH_THIN
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(yd, 'id', ['GrantCount', 'GrantRate', 'StateName', 'Party'])
    ).transform_filter(
        "(party_filter == 'All') || (datum.Party == party_filter)"
    ).transform_filter(
        f"year_select == {year}"
    ).encode(
        color=alt.Color('GrantRate:Q', scale=color_definition, legend=None),
        opacity=alt.condition(f'year_select == {year}', alt.value(OPACITY_ACTIVE), alt.value(OPACITY_INACTIVE)),
        tooltip=[
            'StateName:N',
            'Party:N',
            alt.Tooltip('GrantRate:Q', format='.2f', title='GrantRate (%)'),
            alt.Tooltip('GrantCount:Q', format=',')
        ]
    ).project('albersUsa')

    layers.append(map_shape)

    show_legend = alt.Legend(
        title='Party',
        orient='top-left',
        offset=PARTY_LEGEND_OFFSET_Y
    ) if i == 0 else None

    map_dots = alt.Chart(yd).mark_circle(
        size=CIRCLE_SIZE,
        stroke=COLOR_STROKE_BLACK,
        strokeWidth=STROKE_WIDTH_THIN
    ).transform_filter(
        "(party_filter == 'All') || (datum.Party == party_filter)"
    ).transform_filter(
        f"year_select == {year}"
    ).encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        color=alt.Color('Party:N', scale=party_color_scale, legend=show_legend),
        opacity=alt.condition(f'year_select == {year}', alt.value(OPACITY_CIRCLE), alt.value(OPACITY_INACTIVE)),
        tooltip=['StateName:N', 'Party:N', alt.Tooltip('GrantRate:Q', format='.2f', title='GrantRate (%)')]
    ).project('albersUsa')

    layers.append(map_dots)

map_main = (background + alt.layer(*layers)).properties(width=MAP_WIDTH, height=MAP_HEIGHT)

# CUSTOM LEGEND (FOR MAP)

legend_steps = 300 # Dummy data
legend_values = np.linspace(0, max_rate if max_rate > 0 else 1, legend_steps)
legend_df = pd.DataFrame({"value": legend_values, "y": 1})

l_title = alt.Chart(pd.DataFrame({'text': ['Grant Share (%)']})).mark_text(
    fontWeight='bold', dy=-5
).encode(text='text').properties(width=LEGEND_WIDTH, height=LEGEND_TEXT_HEIGHT)

# Gradient
l_rect = alt.Chart(legend_df).mark_rect().encode(
    x=alt.X("value:Q", bin=alt.Bin(maxbins=legend_steps), axis=None, scale=alt.Scale(domain=[0, max_rate])),
    y=alt.Y("y:O", axis=None),
    color=alt.Color("value:Q", scale=color_definition, legend=None)
).properties(width=LEGEND_WIDTH, height=LEGEND_BAR_HEIGHT)

# Ticks
l_ticks = alt.Chart(pd.DataFrame({"value": np.linspace(0, max_rate, 6)})).mark_text(dy=5).encode(
    x=alt.X("value:Q", scale=alt.Scale(domain=[0, max_rate]), axis=None),
    text=alt.Text("value:Q", format=".1f")
).properties(width=LEGEND_WIDTH, height=LEGEND_BAR_HEIGHT)

legend_col = alt.vconcat(l_title, l_rect, l_ticks, spacing=2)

spacer = alt.Chart(pd.DataFrame({'x': [0]})).mark_point(opacity=0).encode().properties(
    width=LEGEND_SPACER_WIDTH, height=1
)

legend_centered = alt.hconcat(spacer, legend_col, spacer, spacing=0)

right_side = alt.vconcat(map_main, legend_centered, spacing=10).resolve_scale(color='independent')

# FINAL CHART

q1_chart = alt.hconcat(
    bar_chart,
    right_side
).add_params(
    year_param,
    topn_param,
    party_param
).properties(
    title=alt.TitleParams(
        text='NSF Grant Distribution by State',
        subtitle='Percentage of total grants (2021-2025), colored by political party. Includes cancelled and terminated grants',
        anchor='middle',
        offset=20,
        fontSize=18,
        subtitleFontSize=12,
        subtitlePadding=10
    )
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None,
)

q1_chart.display(renderer='svg', embed_options={'actions': False}) # to remove border and cleaner version

alt.HConcatChart(...)

<br></br>

## **How are the grants distributed per directorates?**

In [10]:
df_terminated = df_cancellations[df_cancellations['Status'] == 'Terminated'].copy()
df_reinstated = df_cancellations[df_cancellations['Status'] == 'Reinstated'].copy()

# Directorate aggregation (All years, yearly average)
dir_full_all = df_awards.groupby(['DirectorateAbbr', 'Directorate'])['AwardID'].count().reset_index(name='TotalGrants')
dir_term_all = df_terminated.groupby('DirectorateAbbr')['AwardID'].count().reset_index(name='Terminated')
dir_reinst_all = df_reinstated.groupby('DirectorateAbbr')['AwardID'].count().reset_index(name='Reinstated')

df_dir_all = dir_full_all.merge(dir_term_all, on='DirectorateAbbr', how='left')
df_dir_all = df_dir_all.merge(dir_reinst_all, on='DirectorateAbbr', how='left')

df_dir_all['Terminated'] = df_dir_all['Terminated'].fillna(0)
df_dir_all['Reinstated'] = df_dir_all['Reinstated'].fillna(0)
df_dir_all['Cancelled'] = df_dir_all['Terminated'] + df_dir_all['Reinstated']
df_dir_all['TotalGrants'] = df_dir_all['TotalGrants'] / NUM_YEARS
df_dir_all['CancelRate'] = (df_dir_all['Cancelled'] / (df_dir_all['TotalGrants'] * NUM_YEARS)) * 100
df_dir_all['TerminationRate'] = df_dir_all.apply(
    lambda r: (r['Terminated'] / r['Cancelled'] * 100) if r['Cancelled'] > 0 else 0, axis=1
)
df_dir_all['Year'] = YEAR_ALL_INDICATOR

# Directorate aggregation (By year)
dir_full_year = df_awards.groupby(['DirectorateAbbr', 'Directorate', 'Year'])['AwardID'].count().reset_index(name='TotalGrants')
dir_term_year = df_terminated.groupby(['DirectorateAbbr', 'Year'])['AwardID'].count().reset_index(name='Terminated')
dir_reinst_year = df_reinstated.groupby(['DirectorateAbbr', 'Year'])['AwardID'].count().reset_index(name='Reinstated')

df_dir_year = dir_full_year.merge(dir_term_year, on=['DirectorateAbbr', 'Year'], how='left')
df_dir_year = df_dir_year.merge(dir_reinst_year, on=['DirectorateAbbr', 'Year'], how='left')

df_dir_year['Terminated'] = df_dir_year['Terminated'].fillna(0)
df_dir_year['Reinstated'] = df_dir_year['Reinstated'].fillna(0)
df_dir_year['Cancelled'] = df_dir_year['Terminated'] + df_dir_year['Reinstated']
df_dir_year['CancelRate'] = (df_dir_year['Cancelled'] / df_dir_year['TotalGrants']) * 100
df_dir_year['TerminationRate'] = df_dir_year.apply(
    lambda r: (r['Terminated'] / r['Cancelled'] * 100) if r['Cancelled'] > 0 else 0, axis=1
)

# Combine directorates
df_scatter = pd.concat([df_dir_all, df_dir_year], ignore_index=True)

# Division aggregation (All years, yearly average)
div_full_all = df_awards.groupby(['DirectorateAbbr', 'Directorate', 'DivisionAbbr', 'Division'])['AwardID'].count().reset_index(name='TotalGrants')
div_term_all = df_terminated.groupby(['DirectorateAbbr', 'DivisionAbbr'])['AwardID'].count().reset_index(name='Terminated')
div_reinst_all = df_reinstated.groupby(['DirectorateAbbr', 'DivisionAbbr'])['AwardID'].count().reset_index(name='Reinstated')

df_div_all = div_full_all.merge(div_term_all, on=['DirectorateAbbr', 'DivisionAbbr'], how='left')
df_div_all = df_div_all.merge(div_reinst_all, on=['DirectorateAbbr', 'DivisionAbbr'], how='left')

df_div_all['Terminated'] = df_div_all['Terminated'].fillna(0)
df_div_all['Reinstated'] = df_div_all['Reinstated'].fillna(0)
df_div_all['Cancelled'] = df_div_all['Terminated'] + df_div_all['Reinstated']
df_div_all['TotalGrants'] = df_div_all['TotalGrants'] / NUM_YEARS
df_div_all['CancelRate'] = (df_div_all['Cancelled'] / (df_div_all['TotalGrants'] * NUM_YEARS)) * 100
df_div_all['TerminationRate'] = df_div_all.apply(
    lambda r: (r['Terminated'] / r['Cancelled'] * 100) if r['Cancelled'] > 0 else 0, axis=1
)
df_div_all['Year'] = YEAR_ALL_INDICATOR

# Division aggregation (By year)
div_full_year = df_awards.groupby(['DirectorateAbbr', 'Directorate', 'DivisionAbbr', 'Division', 'Year'])['AwardID'].count().reset_index(name='TotalGrants')
div_term_year = df_terminated.groupby(['DirectorateAbbr', 'DivisionAbbr', 'Year'])['AwardID'].count().reset_index(name='Terminated')
div_reinst_year = df_reinstated.groupby(['DirectorateAbbr', 'DivisionAbbr', 'Year'])['AwardID'].count().reset_index(name='Reinstated')

df_div_year = div_full_year.merge(div_term_year, on=['DirectorateAbbr', 'DivisionAbbr', 'Year'], how='left')
df_div_year = df_div_year.merge(div_reinst_year, on=['DirectorateAbbr', 'DivisionAbbr', 'Year'], how='left')

df_div_year['Terminated'] = df_div_year['Terminated'].fillna(0)
df_div_year['Reinstated'] = df_div_year['Reinstated'].fillna(0)
df_div_year['Cancelled'] = df_div_year['Terminated'] + df_div_year['Reinstated']
df_div_year['CancelRate'] = (df_div_year['Cancelled'] / df_div_year['TotalGrants']) * 100
df_div_year['TerminationRate'] = df_div_year.apply(
    lambda r: (r['Terminated'] / r['Cancelled'] * 100) if r['Cancelled'] > 0 else 0, axis=1
)

# Combine divisions
df_div = pd.concat([df_div_all, df_div_year], ignore_index=True)

In [11]:
# CONSTANTS TO ADJUST THE CHART

CHART_WIDTH_BUBBLE = 700
CHART_HEIGHT_BUBBLE = 400
CHART_WIDTH_BAR = 320
CHART_WIDTH_LABEL = 40
CHART_HEIGHT_BAR = 300
HEADER_HEIGHT = 30
FOOTER_HEIGHT = 20

COLOR_VOLUME = '#00897B'
COLOR_IMPACT = '#AD1457'
COLOR_GRAY = 'lightgray'

PALETTE_DIRECTORATES = [
    '#E69F00', '#56B4E9', '#8D6E63', '#F0E442', '#0072B2',
    '#D55E00', '#CC79A7', '#999999', '#000000'
]

STEP_BUBBLE_X = 200
STEP_BAR_X = 500
STEP_RATE = 2
MIN_BAR_GRANTS = 25
MIN_BAR_RATE = 0

df_dummy = pd.DataFrame({'dummy': [1]}) # Placeholder for text layers that don't use real data

max_grants_global = df_scatter['TotalGrants'].max()
max_domain_bubble = math.ceil(max_grants_global / STEP_BUBBLE_X) * STEP_BUBBLE_X

cancel_rate_min = df_scatter['CancelRate'].min()
cancel_rate_max = df_scatter['CancelRate'].max()
padding_y = (cancel_rate_max - cancel_rate_min) * 0.25
max_domain_rate_bubble = cancel_rate_max + padding_y

max_domain_vol_bar = max_domain_bubble
max_rate_global = max(df_scatter['CancelRate'].max(), df_div['CancelRate'].max())
max_domain_rate_bar = math.ceil(max_rate_global / 2) * 2

ticks_vol = list(range(0, int(max_domain_vol_bar) + 1, STEP_BAR_X))
ticks_rate = list(range(0, int(max_domain_rate_bar) + 1, STEP_RATE))
ticks_bubble_y = [i for i in range(0, int(max_domain_rate_bubble) + 1, 2)]

click_selection = alt.selection_point(fields=['DirectorateAbbr'], name='dir_select')

year_labels_q2 = ['All', '2021', '2022', '2023', '2024', '2025']
year_dropdown = alt.binding_select(options=year_options, labels=year_labels_q2, name='Fiscal Year: ')
year_select = alt.param(name='year_param', value=YEAR_ALL_INDICATOR, bind=year_dropdown)

sort_order = alt.EncodingSortField(field='TotalGrants', op='max', order='descending')

MODE_DEFAULT = "!length(data('dir_select_store'))"
MODE_DRILLDOWN = "length(data('dir_select_store'))"

# BUBBLE CHART
bubble_layer = alt.Chart(df_scatter).mark_circle(
    opacity=0.8, stroke='white', strokeWidth=2
).encode(
    x=alt.X('TotalGrants:Q', title=None,
            scale=alt.Scale(domainMin=0, domainMax=max_domain_bubble, nice=False)),
    y=alt.Y('CancelRate:Q', title='Cancellation Rate (%)',
            scale=alt.Scale(domain=[-2, max_domain_rate_bubble], nice=False),
            axis=alt.Axis(values=ticks_bubble_y)),
        size=alt.Size('TerminationRate:Q',
              scale=alt.Scale(domain=[0, 100], range=[200, 2000]),
              legend=alt.Legend(orient='none', legendX=720, legendY=175,
                  title=['Termination Rate (%)'],
                  symbolFillColor='gray', values=[0, 25, 50, 75, 100])),
    color=alt.condition(click_selection,
        alt.Color('DirectorateAbbr:N', legend=None,
                  scale=alt.Scale(range=PALETTE_DIRECTORATES), sort=sort_order),
        alt.value(COLOR_GRAY)),
    order=alt.Order('TotalGrants:Q', sort='descending'),
    tooltip=[
        alt.Tooltip('Directorate:N', title='Directorate'),
        alt.Tooltip('DirectorateAbbr:N', title='Abbr'),
        alt.Tooltip('TotalGrants:Q', title='Total Grants'),
        alt.Tooltip('CancelRate:Q', title='Cancellation Rate (%)', format='.2f'),
        alt.Tooltip('TerminationRate:Q', title='Termination Rate (%)', format='.2f')
    ]
).add_params(click_selection, year_select
).transform_filter('datum.Year == year_param'
).properties(width=CHART_WIDTH_BUBBLE, height=CHART_HEIGHT_BUBBLE)

# Change title for ALL
bubble_axis_title_avg = alt.Chart(df_dummy).mark_text(
    dy=35, fontWeight='bold', text='Total Grants (Yearly Avg)'
).encode(
    x=alt.value(CHART_WIDTH_BUBBLE / 2), y=alt.value(CHART_HEIGHT_BUBBLE)
).transform_filter(f'year_param == {YEAR_ALL_INDICATOR}')

bubble_axis_title_norm = alt.Chart(df_dummy).mark_text(
    dy=35, fontWeight='bold', text='Total Grants'
).encode(
    x=alt.value(CHART_WIDTH_BUBBLE / 2), y=alt.value(CHART_HEIGHT_BUBBLE)
).transform_filter(f'year_param != {YEAR_ALL_INDICATOR}')

# Directorate legend
legend_directorate = alt.Chart(df_scatter).mark_circle(size=100).encode(
    y=alt.Y('DirectorateAbbr:N',
            axis=alt.Axis(orient='right', title=None, domain=False, ticks=False),
            sort=sort_order, scale=alt.Scale(padding=0)),
    color=alt.Color('DirectorateAbbr:N', legend=None,
                    scale=alt.Scale(range=PALETTE_DIRECTORATES), sort=sort_order)
).transform_filter('datum.Year == year_param'
).properties(width=50, height=150,
    title=alt.TitleParams(text='Directorate', anchor='start', fontSize=11, dx=10))

bubble_with_titles = alt.layer(bubble_layer, bubble_axis_title_avg, bubble_axis_title_norm)

top_sect = alt.hconcat(bubble_with_titles, legend_directorate, spacing=15
).resolve_scale(color='shared')

# Butterfly headers
head_vol_dir = alt.Chart(df_dummy).mark_text(fontWeight='bold', fontSize=12, dy=10, text='Volume (Directorates)')\
    .transform_filter(MODE_DEFAULT)
head_vol_div = alt.Chart(df_dummy).mark_text(fontWeight='bold', fontSize=12, dy=10, text='Volume (Divisions)')\
    .transform_filter(MODE_DRILLDOWN)
header_vol_layer = alt.layer(head_vol_dir, head_vol_div).properties(width=CHART_WIDTH_BAR, height=HEADER_HEIGHT)

head_risk_dir = alt.Chart(df_dummy).mark_text(fontWeight='bold', fontSize=12, dy=10, text='Impact (Directorates)')\
    .transform_filter(MODE_DEFAULT)
head_risk_div = alt.Chart(df_dummy).mark_text(fontWeight='bold', fontSize=12, dy=10, text='Impact (Divisions)')\
    .transform_filter(MODE_DRILLDOWN)
header_risk_layer = alt.layer(head_risk_dir, head_risk_div).properties(width=CHART_WIDTH_BAR, height=HEADER_HEIGHT)

header_spacer = alt.Chart(df_dummy).mark_text(text='').properties(width=CHART_WIDTH_LABEL, height=HEADER_HEIGHT)

# DEFAULT VIEW (Directorate)
base_def = alt.Chart(df_scatter).transform_filter('datum.Year == year_param').transform_filter(MODE_DEFAULT)

def_left = base_def.mark_bar(color=COLOR_VOLUME).encode(
    y=alt.Y('DirectorateAbbr:N', axis=None, sort=sort_order),
    x=alt.X('TotalGrants:Q', sort='descending',
            scale=alt.Scale(domainMin=0, domainMax=max_domain_vol_bar, nice=False),
            axis=alt.Axis(values=ticks_vol, title=None)),
    tooltip=[
        alt.Tooltip('Directorate:N', title='Directorate'),
        alt.Tooltip('DirectorateAbbr:N', title='Abbr'),
        alt.Tooltip('TotalGrants:Q', title='Total Grants')
    ]
).properties(width=CHART_WIDTH_BAR, height=CHART_HEIGHT_BAR)

def_mid = base_def.mark_text().encode(
    y=alt.Y('DirectorateAbbr:N', axis=None, sort=sort_order),
    text='DirectorateAbbr:N'
).properties(width=CHART_WIDTH_LABEL, height=CHART_HEIGHT_BAR)

def_right = base_def.mark_bar(color=COLOR_IMPACT).encode(
    y=alt.Y('DirectorateAbbr:N', axis=None, sort=sort_order),
    x=alt.X('CancelRate:Q', title='Cancellation Rate (%)',
            scale=alt.Scale(domainMin=0, domainMax=max_domain_rate_bar, nice=False),
            axis=alt.Axis(values=ticks_rate, titlePadding=21)),
    tooltip=[
        alt.Tooltip('Directorate:N', title='Directorate'),
        alt.Tooltip('DirectorateAbbr:N', title='Abbr'),
        alt.Tooltip('CancelRate:Q', title='Cancellation Rate (%)', format='.2f')
    ]
).properties(width=CHART_WIDTH_BAR, height=CHART_HEIGHT_BAR)

# DRILL-DOWN VIEW (Divisions)
base_drill = alt.Chart(df_div).transform_filter('datum.Year == year_param')\
    .transform_filter(MODE_DRILLDOWN).transform_filter(click_selection)\
    .transform_filter(alt.datum.TotalGrants > 0)\
    .transform_calculate(
        TotalGrantsDisplay=f"max(datum.TotalGrants, {MIN_BAR_GRANTS})",
        CancelRateDisplay=f"max(datum.CancelRate, {MIN_BAR_RATE})")

drill_left = base_drill.mark_bar(color=COLOR_VOLUME).encode(
    y=alt.Y('DivisionAbbr:N', axis=None, sort=sort_order),
    x=alt.X('TotalGrantsDisplay:Q', sort='descending',
            scale=alt.Scale(domainMin=0, domainMax=max_domain_vol_bar, nice=False),
            axis=alt.Axis(values=ticks_vol, title=None)),
    tooltip=[
        alt.Tooltip('Division:N', title='Division'),
        alt.Tooltip('DivisionAbbr:N', title='Abbr'),
        alt.Tooltip('TotalGrants:Q', title='Total Grants', format=',.1f')
    ]
).properties(width=CHART_WIDTH_BAR, height=CHART_HEIGHT_BAR)

drill_mid = base_drill.mark_text().encode(
    y=alt.Y('DivisionAbbr:N', axis=None, sort=sort_order),
    text='DivisionAbbr:N'
).properties(width=CHART_WIDTH_LABEL, height=CHART_HEIGHT_BAR)

drill_right = base_drill.mark_bar(color=COLOR_IMPACT).encode(
    y=alt.Y('DivisionAbbr:N', axis=None, sort=sort_order),
    x=alt.X('CancelRateDisplay:Q',
            scale=alt.Scale(domainMin=0, domainMax=max_domain_rate_bar, nice=False),
            axis=alt.Axis(values=ticks_rate, title='Cancellation Rate (%)', titlePadding=21)),
    tooltip=[
        alt.Tooltip('Division:N', title='Division'),
        alt.Tooltip('DivisionAbbr:N', title='Abbr'),
        alt.Tooltip('CancelRate:Q', title='Cancellation Rate (%)', format='.2f')
    ]
).properties(width=CHART_WIDTH_BAR, height=CHART_HEIGHT_BAR)

# FOOTER
foot_avg = alt.Chart(df_dummy).mark_text(fontWeight='bold', fontSize=11, color='black', dy=-15, text='Total Grants (Yearly Avg)')\
    .transform_filter(f'year_param == {YEAR_ALL_INDICATOR}')
foot_norm = alt.Chart(df_dummy).mark_text(fontWeight='bold', fontSize=11, color='black', dy=-15, text='Total Grants')\
    .transform_filter(f'year_param != {YEAR_ALL_INDICATOR}')
footer_layer = alt.layer(foot_avg, foot_norm).properties(width=CHART_WIDTH_BAR, height=FOOTER_HEIGHT)

col1 = alt.vconcat(header_vol_layer, alt.layer(def_left, drill_left), footer_layer).resolve_scale(x='shared')
col2 = alt.vconcat(header_spacer, alt.layer(def_mid, drill_mid), header_spacer.properties(height=FOOTER_HEIGHT))
col3 = alt.vconcat(header_risk_layer, alt.layer(def_right, drill_right))

bottom_sect = alt.hconcat(col1, col2, col3, spacing=10).resolve_scale(y='independent')

q2_chart = alt.vconcat(top_sect, bottom_sect).properties(
    title=alt.TitleParams(
        text='NSF Grant Distribution and Cancellation Rate by Directorate',
        subtitle=['Click a directorate bubble to drill-down into divisions. Only major directorates (≥100 total grants across 2021-2025)'],
        anchor='middle', offset=10, fontSize=18, subtitleFontSize=12, subtitlePadding=10
    )
)

q2_chart.configure_view(stroke=None
).configure_title(fontSize=18
).configure_axis(domain=False, domainWidth=0, domainColor='transparent'
).display(renderer='svg', embed_options={'actions': False})

alt.VConcatChart(...)

<br></br>

## **How have the total grants amount evolved over the years?**

In [12]:
# Budget
budget_columns = ['AwardID', 'Year', 'StateCode', 'EstimatedBudget']
df_budget = pd.concat([
    df_awards[budget_columns],
    missing_cancellations[budget_columns]
], ignore_index=True)

# Party
df_budget['Party'] = df_budget.apply(
    lambda row: state_party_2020_map.get(row['StateCode'], 'Unknown') if row['Year'] <= 2024
    else state_party_2025_map.get(row['StateCode'], 'Unknown'), axis=1
)

# Aggregate by Year
yearly_totals = df_budget.groupby('Year').agg(TotalBudget=('EstimatedBudget', 'sum')).reset_index()
yearly_totals['Group'] = 'All'

yearly_party_totals = df_budget.groupby(['Year', 'Party']).agg(TotalBudget=('EstimatedBudget', 'sum')).reset_index()
yearly_party_totals['Group'] = yearly_party_totals['Party']

combined_q4 = pd.concat([yearly_totals, yearly_party_totals], ignore_index=True)

In [15]:
q4_y_max = combined_q4['TotalBudget'].max() * 1.05
q4_y_domain = [0, q4_y_max / 1e9]

party_options = ['Total', 'Democrat', 'Republican']
party_param = alt.param(
    name='party_filter',
    value='Total',
    bind=alt.binding_select(options=party_options, name='Party: ')
)

year_click_selection = alt.selection_point(fields=['Year'], on='click', clear='dblclick')

q4_color_scale = alt.Scale(domain=['Total', 'Democrat', 'Republican'], range=['#7B1FA2', COLOR_DEMOCRAT, COLOR_REPUBLICAN])

q4_base = alt.Chart(combined_q4).transform_calculate(
    DisplayGroup="datum.Group == 'All' ? 'Total' : datum.Group"
).transform_filter(
    "datum.DisplayGroup == party_filter"
)

q4_line = q4_base.transform_calculate(
    TotalBudgetBillions='datum.TotalBudget / 1000000000'
).mark_line(strokeWidth=2).encode(
    x=alt.X('Year:O', axis=alt.Axis(labelAngle=0, title='Fiscal Year', labelOffset=10, labelPadding=8)),
    y=alt.Y('TotalBudgetBillions:Q', title='Total Budget (Billions $)', scale=alt.Scale(domain=q4_y_domain)),
    color=alt.Color('DisplayGroup:N', scale=q4_color_scale, legend=alt.Legend(title='State Party')),
    opacity=alt.value(0.6)
)

q4_points = q4_base.transform_calculate(
    TotalBudgetBillions='datum.TotalBudget / 1000000000'
).mark_circle(size=80).encode(
    x=alt.X('Year:O'),
    y=alt.Y('TotalBudgetBillions:Q', scale=alt.Scale(domain=q4_y_domain)),
    color=alt.Color('DisplayGroup:N', scale=q4_color_scale, legend=None),
    opacity=alt.condition(year_click_selection, alt.value(1), alt.value(0.4)),
    size=alt.condition(year_click_selection, alt.value(80), alt.value(50)),
    tooltip=[alt.Tooltip('Year:O'), alt.Tooltip('DisplayGroup:N', title='Party'), alt.Tooltip('TotalBudget:Q', format='$,.0f')]
).add_params(party_param, year_click_selection)

q4_rule = q4_base.transform_filter(year_click_selection).mark_rule(
    strokeDash=[5, 5], color='gray', opacity=0.8
).encode(x=alt.X('Year:O'))

# Annotation for 2025 Trump Cancellations
df_annotation_2025 = pd.DataFrame({'Year': [2025]})

q4_annotation_rule = alt.Chart(df_annotation_2025).mark_rule(
    color='#666666', strokeWidth=2, strokeDash=[4, 2]
).encode(x=alt.X('Year:O'))

q4_annotation_text = alt.Chart(df_annotation_2025).mark_text(
    align='left', dx=5, dy=48, fontSize=11, fontWeight='bold', color='#666666'
).encode(
    x=alt.X('Year:O'),
    text=alt.value('Trump Cancellations')
)

q4_chart = (q4_annotation_rule + q4_line + q4_points + q4_rule + q4_annotation_text).properties(
    title=alt.TitleParams(
        text='NSF Grants Budget Evolution Over Time',
        subtitle='Click a point to select. Budget allocated to the year the grant was approved'
    ),
    width=600,
    height=300
)

q4_chart.display(renderer='svg', embed_options={'actions': False})

alt.LayerChart(...)

<br></br>

## **For a selected state, how have the grants evolved?**

In [16]:
df_cancellations_with_year = df_cancellations.merge(
    df_awards[['AwardID', 'Year', 'StateName', 'StateCode']].drop_duplicates(),
    on='AwardID',
    how='left',
    suffixes=('', '_awards')
)

df_cancellations_with_year['StateCode'] = df_cancellations_with_year['StateCode'].fillna(df_cancellations_with_year.get('StateCode_awards', ''))
df_cancellations_with_year['StateName'] = df_cancellations_with_year['StateName'].fillna(df_cancellations_with_year.get('StateName_awards', ''))

cancelled_by_state_year = df_cancellations_with_year.groupby(
    ['Year', 'StateCode', 'StateName', 'Status']
).size().reset_index(name='Count')

In [17]:
all_grants_total = df_state_grants.groupby(['Year']).agg(
    GrantCount=('GrantCount', 'mean')
).reset_index()
all_grants_total['StateName'] = 'All (Average)'
all_grants_total['StateCode'] = 'ALL'
all_grants_total['Party'] = 'All'

df_state_grants_with_all = pd.concat([
    df_state_grants,
    all_grants_total[['Year', 'StateName', 'StateCode', 'Party', 'GrantCount']]
], ignore_index=True)

segments_list = []
for state in df_state_grants_with_all['StateName'].unique():
    state_data = df_state_grants_with_all[df_state_grants_with_all['StateName'] == state].sort_values('Year')
    for i in range(len(state_data) - 1):
        row1 = state_data.iloc[i]
        row2 = state_data.iloc[i + 1]
        segments_list.append({
            'StateName': state,
            'Year1': row1['Year'],
            'Year2': row2['Year'],
            'GrantCount1': row1['GrantCount'],
            'GrantCount2': row2['GrantCount'],
            'Party': row2['Party']
        })

df_segments = pd.DataFrame(segments_list)

all_cancelled_avg = cancelled_by_state_year.groupby(['Year', 'Status']).agg(
    Count=('Count', 'mean')
).reset_index()
all_cancelled_avg['StateName'] = 'All (Average)'
all_cancelled_avg['StateCode'] = 'ALL'

cancelled_with_all = pd.concat([
    cancelled_by_state_year,
    all_cancelled_avg[['Year', 'StateCode', 'StateName', 'Status', 'Count']]
], ignore_index=True)

In [19]:
q5_y_max = df_state_grants_with_all['GrantCount'].max() * 1.05
q5_y_domain = [0, q5_y_max]

state_list = ['All (Average)'] + sorted(df_state_grants['StateName'].unique().tolist())
state_dropdown = alt.binding_select(options=state_list, name='State: ')
state_selection = alt.selection_point(fields=['StateName'], bind=state_dropdown, value='All (Average)')

year_click_q5 = alt.selection_point(fields=['Year'], on='click', clear='dblclick', empty=True)

q5_color_scale = alt.Scale(domain=['All', 'Democrat', 'Republican'], range=['#7B1FA2', COLOR_DEMOCRAT, COLOR_REPUBLICAN])
status_color_scale = alt.Scale(domain=['Terminated', 'Reinstated'], range=['#424242', '#FF8A65'])

max_cancelled_count = cancelled_with_all.groupby(['Year', 'StateName'])['Count'].sum().max()
q5_2_x_domain = [0, max_cancelled_count * 1.1 if max_cancelled_count > 0 else 5]

# 2025 Annotation data
df_annotation_2025 = pd.DataFrame({'Year': [2025]})

# Part 1
q5_base = alt.Chart(df_state_grants_with_all).transform_filter(state_selection)
q5_segments_base = alt.Chart(df_segments).transform_filter(state_selection)

q5_annotation_rule = alt.Chart(df_annotation_2025).mark_rule(
    color='#666666', strokeWidth=2, strokeDash=[4, 2]
).encode(x=alt.X('Year:O')).transform_filter(year_click_q5)

q5_annotation_text = alt.Chart(df_annotation_2025).mark_text(
    align='left', dx=5, dy=48, fontSize=11, fontWeight='bold', color='#666666'
).encode(x=alt.X('Year:O'), text=alt.value('Trump Cancellations')).transform_filter(year_click_q5)

# Line segments colored by destination year's party
q5_line = q5_segments_base.mark_rule(strokeWidth=2).encode(
    x=alt.X('Year1:O', title='Fiscal Year', axis=alt.Axis(labelAngle=0, labelOffset=10, labelPadding=10)),
    x2='Year2:O',
    y=alt.Y('GrantCount1:Q', title='Number of Grants', scale=alt.Scale(domain=q5_y_domain)),
    y2='GrantCount2:Q',
    color=alt.Color('Party:N', scale=q5_color_scale, legend=alt.Legend(title='State Party')),
    opacity=alt.value(0.7)
).add_params(state_selection, year_click_q5)

# Points with actual Party color
q5_points = q5_base.mark_circle(size=80).encode(
    x='Year:O', y=alt.Y('GrantCount:Q', scale=alt.Scale(domain=q5_y_domain)),
    color=alt.Color('Party:N', scale=q5_color_scale, legend=None),
    opacity=alt.condition(year_click_q5, alt.value(1), alt.value(0.7)),
    size=alt.condition(year_click_q5, alt.value(100), alt.value(80)),
    tooltip=[alt.Tooltip('Year:O'), alt.Tooltip('StateName:N'), alt.Tooltip('Party:N'), alt.Tooltip('GrantCount:Q', format='.1f')]
)

q5_rule = q5_base.transform_filter(year_click_q5).mark_rule(strokeDash=[5, 5], color='gray', opacity=0.8).encode(x='Year:O')

q5_part1_chart = (q5_annotation_rule + q5_line + q5_points + q5_rule + q5_annotation_text).properties(
    title='NSF Grants Evolution for Selected State', width=600, height=300
)

# Part 2
q5_part2_chart = alt.Chart(cancelled_with_all).mark_bar().encode(
    y=alt.Y('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),
    x=alt.X('Count:Q', title='Cancelled Grants', stack='zero', scale=alt.Scale(domain=q5_2_x_domain)),
    color=alt.Color('Status:N', scale=status_color_scale, legend=alt.Legend(title='Status')),
    tooltip=[alt.Tooltip('Year:O'), alt.Tooltip('StateName:N'), alt.Tooltip('Status:N'), alt.Tooltip('Count:Q', format='.1f')]
).transform_filter(state_selection).transform_filter(year_click_q5).properties(
    title={'text': 'NSF Cancelled Grants by Status', 'subtitle': 'Click year above to filter'}, width=600, height=200
)

q5_chart = alt.vconcat(q5_part1_chart, q5_part2_chart).resolve_scale(color='independent').configure_view(stroke=None)
q5_chart.display(renderer='svg', embed_options={'actions': False})

alt.VConcatChart(...)

<br></br>

## **Final Visualization**

In [20]:
# CONSTANTS

# Chart Dimensions
MAP_WIDTH = 470
MAP_HEIGHT = 300
MAP_SCALE = int(1.30 * MAP_WIDTH)

BAR_WIDTH = 250
BAR_HEIGHT = 330
BAR_CHART_TOP_PADDING = 36

Q5_BAR_HEIGHT = 150
Q5_WIDTH = BAR_WIDTH + MAP_WIDTH
Q5_2_WIDTH = MAP_WIDTH + BAR_WIDTH - 20

LINE_CHART_WIDTH = 300
LINE_CHART_HEIGHT = 275

# Legend
LEGEND_WIDTH = MAP_WIDTH - 50
LEGEND_SPACER_WIDTH = (MAP_WIDTH - LEGEND_WIDTH) / 2
LEGEND_TEXT_HEIGHT = 20
LEGEND_BAR_HEIGHT = 15
LEGEND_STEPS = 300
LEGEND_TICK_COUNT = 6

# Q2 Bubble & Bar
CHART_WIDTH_BUBBLE = 450
CHART_HEIGHT_BUBBLE = 400
CHART_WIDTH_BAR = 275
CHART_WIDTH_LABEL = 40
CHART_HEIGHT_BAR = 350
HEADER_HEIGHT = 30
FOOTER_HEIGHT = 20
BUBBLE_HEADER_OFFSET = 4
BUTTERFLY_HEADER_OFFSET = 2

# --- Colors ---
COLOR_DEMOCRAT = "#377eb8"
COLOR_REPUBLICAN = "#e41a1c"
COLOR_DEMOCRAT_DARK = "#12129E"
COLOR_REPUBLICAN_DARK = "#8B0000"
COLOR_STROKE_BLACK = "black"
COLOR_STROKE_WHITE = "white"
COLOR_BACKGROUND_MAP = "#e0e0e0"
COLOR_SCHEME_RATE = "teals"
COLOR_ALL_PARTY = "#7B1FA2"

STROKE_WIDTH_THIN = 0.5
STROKE_WIDTH_THICK = 4
STROKE_WIDTH_SYMBOL = 5

OPACITY_ACTIVE = 1.0
OPACITY_INACTIVE = 0.0
OPACITY_CIRCLE = 0.9
OPACITY_RULE_OUTLINE = 0.8
OPACITY_LINE = 0.6
OPACITY_YEAR_RULE = 0.7

CIRCLE_SIZE = 50
POINT_SIZE_DEFAULT = 80
POINT_SIZE_SELECTED = 100
POINT_SIZE_UNSELECTED = 50

PARTY_LEGEND_OFFSET_Y = -40
MEAN_LEGEND_OFFSET = -30
Q4_HEIGHT_OFFSET = 52

TOP_N_MIN = 5
TOP_N_MAX = 20
TOP_N_STEP = 1
DEFAULT_TOP_N = 10

YEARS_LIST = [2021, 2022, 2023, 2024, 2025]
YEAR_DEFAULT = 2025  # Default year shown when "All Years" is selected

VCONCAT_SPACING = 20
HCONCAT_SPACING = 30
BOTTOM_SPACING = 40
DASHBOARD_TITLE_FONT_SIZE = 20
DASHBOARD_SUBTITLE_FONT_SIZE = 12
DASHBOARD_TITLE_OFFSET = 20

In [21]:
# SHARED INTERACTION PARAMETERS

year_input = alt.binding_select(
    options=[YEAR_ALL_INDICATOR] + YEARS_LIST,
    labels=['All Years (Total)'] + [str(y) for y in YEARS_LIST],
    name='Fiscal Year: '
)
year_param = alt.param(name='year_select', value=YEAR_ALL_INDICATOR, bind=year_input)

topn_slider = alt.binding_range(
    min=TOP_N_MIN, max=TOP_N_MAX, step=TOP_N_STEP, name='Top N States: '
)
topn_param = alt.param(name='topn', value=DEFAULT_TOP_N, bind=topn_slider)

party_input = alt.binding_select(
    options=['All', 'Republican', 'Democrat'],
    name='Party Filter: '
)
party_param = alt.param(name='party_filter', value='All', bind=party_input)

# State selection (map click + dropdown)
state_list = q1_combined['StateName'].unique().tolist()
state_input = alt.binding_select(
    options=[None] + state_list,
    labels=['No State Selected'] + state_list,
    name='State: '
)
state_selection = alt.selection_point(
    fields=['StateName'],
    name='state_click',
    bind=state_input,
    on='click',
    clear='dblclick'
)

# Directorate selection (Q2 drill-down)
dir_selection = alt.selection_point(fields=['DirectorateAbbr'], name='dir_select')

# COLOR SCALES

max_rate = q1_combined['GrantRate'].max()

viridis_scale = alt.Scale(scheme=COLOR_SCHEME_RATE, domain=[0, max_rate])

party_color_scale = alt.Scale(
    domain=["Democrat", "Republican"],
    range=[COLOR_DEMOCRAT, COLOR_REPUBLICAN]
)

party_legend_colors = alt.Scale(
    domain=["Democrat", "Republican"],
    range=[COLOR_DEMOCRAT_DARK, COLOR_REPUBLICAN_DARK]
)

In [22]:
# Q1: BAR CHART + MAP

# Bar Chart
base_bars = alt.Chart(q1_combined).transform_filter(
    alt.datum.Year == year_param
).transform_filter(
    "(party_filter == 'All') || (datum.Party == party_filter)"
).transform_window(
    rank='rank(GrantRate)',
    sort=[alt.SortField('GrantRate', order='descending')]
).transform_filter(
    alt.datum.rank <= topn_param
)

y_axis_config = alt.Y(
    'StateName:N',
    title=None,
    sort=alt.EncodingSortField(field="rank", op="min", order="ascending")
)

bars = base_bars.mark_bar(
    stroke=COLOR_STROKE_BLACK,
    strokeWidth=STROKE_WIDTH_THIN
).encode(
    x=alt.X('GrantRate:Q', title='Grant Share (%)', scale=alt.Scale(domain=[0, max_rate])),
    y=y_axis_config,
    color=alt.Color('Party:N', scale=party_color_scale, legend=None),
    opacity=alt.condition(state_selection, alt.value(OPACITY_ACTIVE), alt.value(0.2)),
    tooltip=['StateName:N', 'Party:N', alt.Tooltip('GrantRate:Q', format='.2f'), alt.Tooltip('GrantCount:Q', format=',')]
).properties(width=BAR_WIDTH, height=BAR_HEIGHT)

# Mean rules
rule_base = alt.Chart(q1_combined).transform_filter(
    alt.datum.Year == year_param
).transform_filter(
    "(party_filter == 'All') || (datum.Party == party_filter)"
).transform_aggregate(
    GrantRate='mean(GrantRate)', groupby=['Party']
)

rule_outline = rule_base.mark_rule(strokeWidth=STROKE_WIDTH_THICK, opacity=OPACITY_RULE_OUTLINE).encode(
    x='GrantRate:Q',
    stroke=alt.value(COLOR_STROKE_BLACK),
    tooltip=[alt.Tooltip('Party:N'), alt.Tooltip('GrantRate:Q', format='.2f', title='Mean')]
)

rule_color = rule_base.mark_rule(strokeWidth=STROKE_WIDTH_SYMBOL).encode(
    x='GrantRate:Q',
    stroke=alt.Stroke('Party:N', scale=party_legend_colors,
        legend=alt.Legend(
            title='Mean by Party',
            orient='top',
            direction='horizontal',
            offset=MEAN_LEGEND_OFFSET,
            symbolStrokeWidth=STROKE_WIDTH_SYMBOL
        ))
)

bar_chart = (bars + rule_outline + rule_color)

# Map
us_states_geo = alt.topo_feature(vega_data.us_10m.url, "states")

background = alt.Chart(us_states_geo).mark_geoshape(
    fill=COLOR_BACKGROUND_MAP,
    stroke=COLOR_STROKE_WHITE,
    strokeWidth=STROKE_WIDTH_THIN
).project(
    'albersUsa',
    scale=MAP_SCALE,
    translate=(MAP_WIDTH // 2, MAP_HEIGHT // 2)
)

layers = []
year_list = [YEAR_ALL_INDICATOR, 2021, 2022, 2023, 2024, 2025]

for i, year in enumerate(year_list):
    yd = q1_combined[q1_combined['Year'] == year].copy()

    is_visible_year = f"year_select == '{year}'" if isinstance(year, str) else f"year_select == {year}"

    map_shape = alt.Chart(us_states_geo).mark_geoshape(
        stroke=COLOR_STROKE_WHITE,
        strokeWidth=STROKE_WIDTH_THIN
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(yd, 'id', ['GrantCount', 'GrantRate', 'StateName', 'Party'])
    ).transform_filter(
        "(party_filter == 'All') || (datum.Party == party_filter)"
    ).transform_filter(
        is_visible_year
    ).encode(
        color=alt.Color('GrantRate:Q', scale=viridis_scale, legend=None),
        opacity=alt.condition(state_selection, alt.value(OPACITY_ACTIVE), alt.value(0.1)),
        tooltip=[
            'StateName:N',
            'Party:N',
            alt.Tooltip('GrantRate:Q', format='.2f'),
            alt.Tooltip('GrantCount:Q', format=',')
        ]
    ).project(
        'albersUsa',
        scale=MAP_SCALE,
        translate=(MAP_WIDTH // 2, MAP_HEIGHT // 2)
    )

    layers.append(map_shape)

    show_legend = alt.Legend(title='Party', orient='top-left', direction='horizontal', offset=PARTY_LEGEND_OFFSET_Y) if i == 0 else None

    map_dots = alt.Chart(yd).mark_circle(
        size=CIRCLE_SIZE,
        stroke=COLOR_STROKE_BLACK,
        strokeWidth=STROKE_WIDTH_THIN
    ).transform_filter(
        "(party_filter == 'All') || (datum.Party == party_filter)"
    ).transform_filter(
        is_visible_year
    ).encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        color=alt.Color('Party:N', scale=party_color_scale, legend=show_legend),
        opacity=alt.condition(state_selection, alt.value(OPACITY_CIRCLE), alt.value(0.1)),
        tooltip=[
            'StateName:N',
            'Party:N',
            alt.Tooltip('GrantRate:Q', format='.2f'),
            alt.Tooltip('GrantCount:Q', format=',')
        ]
    ).project(
        'albersUsa',
        scale=MAP_SCALE,
        translate=(MAP_WIDTH // 2, MAP_HEIGHT // 2)
    )

    layers.append(map_dots)

map_main = (background + alt.layer(*layers)).add_params(
    state_selection
).properties(width=MAP_WIDTH, height=MAP_HEIGHT)

# Legend
legend_values = np.linspace(0, max_rate if max_rate > 0 else 1, LEGEND_STEPS)
legend_df = pd.DataFrame({"value": legend_values, "y": 1})

l_title = alt.Chart(pd.DataFrame({'text': ['Grant Share (%)']})).mark_text(fontWeight='bold', dy=-5).encode(text='text').properties(width=LEGEND_WIDTH, height=LEGEND_TEXT_HEIGHT)

l_rect = alt.Chart(legend_df).mark_rect().encode(
    x=alt.X("value:Q", bin=alt.Bin(maxbins=LEGEND_STEPS), axis=None, scale=alt.Scale(domain=[0, max_rate])),
    y=alt.Y("y:O", axis=None),
    color=alt.Color("value:Q", scale=viridis_scale, legend=None)
).properties(width=LEGEND_WIDTH, height=LEGEND_BAR_HEIGHT)

l_ticks = alt.Chart(pd.DataFrame({"value": np.linspace(0, max_rate, LEGEND_TICK_COUNT)})).mark_text(dy=5).encode(
    x=alt.X("value:Q", scale=alt.Scale(domain=[0, max_rate]), axis=None),
    text=alt.Text("value:Q", format=".1f")
).properties(width=LEGEND_WIDTH, height=LEGEND_BAR_HEIGHT)

legend_col = alt.vconcat(l_title, l_rect, l_ticks, spacing=2)
spacer = alt.Chart(pd.DataFrame({'x': [0]})).mark_point(opacity=0).encode().properties(width=LEGEND_SPACER_WIDTH, height=1)
legend_centered = alt.hconcat(spacer, legend_col, spacer, spacing=0)

# --- Assembly ---
right_side = alt.vconcat(map_main, legend_centered, spacing=10).resolve_scale(color='independent')
q1_row = alt.hconcat(bar_chart, right_side, spacing=0).resolve_scale(color='independent')

In [23]:
# Q5.2: CANCELLED GRANTS BAR CHART

cancelled_by_state_year['StateName'] = cancelled_by_state_year['StateCode'].map(state_name_map)
cancelled_by_state_year['Party'] = cancelled_by_state_year.apply(
    lambda row: state_party_2020_map.get(row['StateCode'], 'Unknown') if row['Year'] <= 2024
    else state_party_2025_map.get(row['StateCode'], 'Unknown'), axis=1
)

q5_2_chart = alt.Chart(cancelled_by_state_year).mark_bar().encode(
    y=alt.Y('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),
    x=alt.X('mean(Count):Q',
            title='Cancelled Grants (Average per State)',
            stack='zero',
            scale=alt.Scale(domain=q5_2_x_domain)),
    color=alt.Color(
        'Status:N',
        scale=status_color_scale,
        legend=alt.Legend(title='Status', orient='top')
    ),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('StateName:N', aggregate='min', title='State'),
        alt.Tooltip('Status:N', title='Status'),
        alt.Tooltip('mean(Count):Q', title='Count (Avg/Actual)', format='.1f')
    ]
).transform_filter(
    state_selection
).transform_filter(
    f"(year_select == '{YEAR_ALL_INDICATOR}') || (datum.Year == year_select)"
).transform_filter(
    "(party_filter == 'All') || (datum.Party == party_filter)"
).properties(
    title={
        'text': 'Cancelled Grants Details',
        'subtitle': 'Average count (All States) or actual (State selected)',
        'anchor': 'start',
        'fontSize': 16
    },
    width=Q5_WIDTH,
    height=Q5_BAR_HEIGHT
)

In [24]:
# Q2: BUBBLE CHART + BUTTERFLY BAR CHART
df_dummy = pd.DataFrame({'dummy': [1]})

# Domain Setup
max_grants_global = df_scatter['TotalGrants'].max()
max_domain_bubble = math.ceil(max_grants_global / STEP_BUBBLE_X) * STEP_BUBBLE_X

cancel_rate_max = df_scatter['CancelRate'].max()
max_domain_rate_bubble = cancel_rate_max + (cancel_rate_max - df_scatter['CancelRate'].min()) * 0.25

max_domain_vol_bar = max_domain_bubble
max_rate_global = max(df_scatter['CancelRate'].max(), df_div['CancelRate'].max())
max_domain_rate_bar = math.ceil(max_rate_global / 2) * 2

ticks_vol = list(range(0, int(max_domain_vol_bar) + 1, STEP_BAR_X))
ticks_rate = list(range(0, int(max_domain_rate_bar) + 1, STEP_RATE))
ticks_bubble_y = [i for i in range(0, int(max_domain_rate_bubble) + 1, 2)]

sort_order = alt.EncodingSortField(field='TotalGrants', op='max', order='descending')
MODE_DEFAULT = "!length(data('dir_select_store'))"
MODE_DRILLDOWN = "length(data('dir_select_store'))"

# Bubble Chart
bubble_layer = alt.Chart(df_scatter).mark_circle(
    opacity=0.8,
    stroke='white',
    strokeWidth=2
).encode(
    x=alt.X(
        'TotalGrants:Q',
        title=None,
        scale=alt.Scale(domainMin=0, domainMax=max_domain_bubble, nice=False)
    ),
    y=alt.Y(
        'CancelRate:Q',
        title='Cancellation Rate (%)',
        scale=alt.Scale(domain=[-2, max_domain_rate_bubble], nice=False),
        axis=alt.Axis(values=ticks_bubble_y)
    ),
    size=alt.Size(
        'TerminationRate:Q',
        scale=alt.Scale(domain=[0, 100], range=[150, 1500]),
        legend=alt.Legend(
            orient='none',
            legendX=CHART_WIDTH_BUBBLE + 20,
            legendY=940,
            title=['Termination Rate (%)'],
            symbolFillColor='gray',
            padding=10,
            cornerRadius=5
        )
    ),
    color=alt.condition(
        dir_selection,
        alt.Color(
            'DirectorateAbbr:N',
            legend=None,
            scale=alt.Scale(range=PALETTE_DIRECTORATES),
            sort=sort_order
        ),
        alt.value(COLOR_GRAY)
    ),
    tooltip=[
        alt.Tooltip('Directorate:N'),
        alt.Tooltip('TotalGrants:Q', format='.1f'),
        alt.Tooltip('CancelRate:Q', format='.1f'),
        alt.Tooltip('TerminationRate:Q', format='.1f')
    ]
).add_params(
    dir_selection
).transform_filter(
    alt.datum.Year == year_param
).properties(
    width=CHART_WIDTH_BUBBLE,
    height=CHART_HEIGHT_BUBBLE
)

bubble_title_avg = alt.Chart(df_dummy).mark_text(
    dy=35,
    fontSize=13,
    fontWeight='normal',
    text='Total Grants (Yearly Avg)'
).encode(
    x=alt.value(CHART_WIDTH_BUBBLE / 2),
    y=alt.value(CHART_HEIGHT_BUBBLE)
).transform_filter(
    f"year_select == '{YEAR_ALL_INDICATOR}'"
)

bubble_title_norm = alt.Chart(df_dummy).mark_text(
    dy=35,
    fontSize=13,
    fontWeight='normal',
    text='Total Grants'
).encode(
    x=alt.value(CHART_WIDTH_BUBBLE / 2),
    y=alt.value(CHART_HEIGHT_BUBBLE)
).transform_filter(
    f"year_select != '{YEAR_ALL_INDICATOR}'"
)

legend_directorate = alt.Chart(df_scatter).mark_circle(size=100).encode(
    y=alt.Y(
        'DirectorateAbbr:N',
        axis=alt.Axis(orient='right', title=None, domain=False, ticks=False),
        sort=sort_order,
        scale=alt.Scale(padding=0)
    ),
    color=alt.Color(
        'DirectorateAbbr:N',
        legend=None,
        scale=alt.Scale(range=PALETTE_DIRECTORATES),
        sort=sort_order
    )
).transform_filter(
    alt.datum.Year == year_param
).properties(
    width=50,
    height=150,
    title=alt.TitleParams(text='Directorate', anchor='start', fontSize=11, dx=10)
)

bubble_header = alt.Chart(df_dummy).mark_text(opacity=0).properties(
    width=CHART_WIDTH_BUBBLE + 65,
    height=BUBBLE_HEADER_OFFSET
)

bubble_footer_avg = alt.Chart(df_dummy).mark_text(
    fontSize=13,
    fontWeight='normal',
    color='#666',
    dy=-15,
    text='Total Grants (Yearly Avg)'
).transform_filter(
    f"year_select == '{YEAR_ALL_INDICATOR}'"
)

bubble_footer_norm = alt.Chart(df_dummy).mark_text(
    fontSize=13,
    fontWeight='normal',
    color='#666',
    dy=-15,
    text='Total Grants'
).transform_filter(
    f"year_select != '{YEAR_ALL_INDICATOR}'"
)

bubble_footer = alt.layer(
    bubble_footer_avg,
    bubble_footer_norm
).properties(
    width=CHART_WIDTH_BUBBLE + 65,
    height=FOOTER_HEIGHT
)

bubble_chart_layer = alt.hconcat(
    bubble_layer,
    legend_directorate,
    spacing=15
).resolve_scale(color='shared')

q2_left_part = alt.vconcat(
    bubble_header,
    bubble_chart_layer,
    bubble_footer
)

# --- Butterfly Bar Chart ---
head_vol_dir = alt.Chart(df_dummy).mark_text(
    fontWeight='bold', fontSize=12, dy=10, text='Volume (Directorates)'
).transform_filter(
    MODE_DEFAULT
)

head_vol_div = alt.Chart(df_dummy).mark_text(
    fontWeight='bold', fontSize=12, dy=10, text='Volume (Divisions)'
).transform_filter(
    MODE_DRILLDOWN
)

header_vol_layer = alt.layer(
    head_vol_dir,
    head_vol_div
).properties(
    width=CHART_WIDTH_BAR,
    height=HEADER_HEIGHT + BUTTERFLY_HEADER_OFFSET
)

head_risk_dir = alt.Chart(df_dummy).mark_text(
    fontWeight='bold', fontSize=12, dy=10, text='Impact (Directorates)'
).transform_filter(
    MODE_DEFAULT
)

head_risk_div = alt.Chart(df_dummy).mark_text(
    fontWeight='bold', fontSize=12, dy=10, text='Impact (Divisions)'
).transform_filter(
    MODE_DRILLDOWN
)

header_risk_layer = alt.layer(
    head_risk_dir,
    head_risk_div
).properties(
    width=CHART_WIDTH_BAR,
    height=HEADER_HEIGHT + BUTTERFLY_HEADER_OFFSET
)

header_spacer = alt.Chart(df_dummy).mark_text(text='').properties(
    width=CHART_WIDTH_LABEL,
    height=HEADER_HEIGHT + BUTTERFLY_HEADER_OFFSET
)

# Default view (Directorates)
base_def = alt.Chart(df_scatter).transform_filter(
    alt.datum.Year == year_param
).transform_filter(MODE_DEFAULT)

def_left = base_def.mark_bar(color=COLOR_VOLUME).encode(
    y=alt.Y('DirectorateAbbr:N', axis=None, sort=sort_order),
    x=alt.X(
        'TotalGrants:Q',
        sort='descending',
        scale=alt.Scale(domainMin=0, domainMax=max_domain_vol_bar, nice=False),
        axis=alt.Axis(values=ticks_vol, title=None)
    ),
    tooltip=[alt.Tooltip('Directorate:N'), alt.Tooltip('TotalGrants:Q', format='.1f')]
).properties(
    width=CHART_WIDTH_BAR,
    height=CHART_HEIGHT_BUBBLE
)

def_mid = base_def.mark_text().encode(
    y=alt.Y('DirectorateAbbr:N', axis=None, sort=sort_order),
    text='DirectorateAbbr:N'
).properties(
    width=CHART_WIDTH_LABEL,
    height=CHART_HEIGHT_BUBBLE
)

def_right = base_def.mark_bar(color=COLOR_IMPACT).encode(
    y=alt.Y('DirectorateAbbr:N', axis=None, sort=sort_order),
    x=alt.X(
        'CancelRate:Q',
        title='Cancellation Rate (%)',
        scale=alt.Scale(domainMin=0, domainMax=max_domain_rate_bar, nice=False),
        axis=alt.Axis(values=ticks_rate, titlePadding=21)
    ),
    tooltip=[alt.Tooltip('Directorate:N'), alt.Tooltip('CancelRate:Q', format='.1f')]
).properties(
    width=CHART_WIDTH_BAR,
    height=CHART_HEIGHT_BUBBLE
)

# Drilldown view (Divisions)
base_drill = alt.Chart(df_div).transform_filter(
    alt.datum.Year == year_param
).transform_filter(
    MODE_DRILLDOWN
).transform_filter(
    dir_selection
).transform_filter(
    alt.datum.TotalGrants > 0
).transform_calculate(
    TotalGrantsDisplay=f"max(datum.TotalGrants, {MIN_BAR_GRANTS})",
    CancelRateDisplay=f"max(datum.CancelRate, {MIN_BAR_RATE})"
)

drill_left = base_drill.mark_bar(color=COLOR_VOLUME).encode(
    y=alt.Y('DivisionAbbr:N', axis=None, sort=sort_order),
    x=alt.X(
        'TotalGrantsDisplay:Q',
        sort='descending',
        scale=alt.Scale(domainMin=0, domainMax=max_domain_vol_bar, nice=False),
        axis=alt.Axis(values=ticks_vol, title=None)
    ),
    tooltip=[alt.Tooltip('Division:N'), alt.Tooltip('TotalGrants:Q', format='.1f')]
).properties(
    width=CHART_WIDTH_BAR,
    height=CHART_HEIGHT_BUBBLE
)

drill_mid = base_drill.mark_text().encode(
    y=alt.Y('DivisionAbbr:N', axis=None, sort=sort_order),
    text='DivisionAbbr:N'
).properties(
    width=CHART_WIDTH_LABEL,
    height=CHART_HEIGHT_BUBBLE
)

drill_right = base_drill.mark_bar(color=COLOR_IMPACT).encode(
    y=alt.Y('DivisionAbbr:N', axis=None, sort=sort_order),
    x=alt.X(
        'CancelRateDisplay:Q',
        scale=alt.Scale(domainMin=0, domainMax=max_domain_rate_bar, nice=False),
        axis=alt.Axis(values=ticks_rate, title='Cancellation Rate (%)', titlePadding=21)
    ),
    tooltip=[alt.Tooltip('Division:N'), alt.Tooltip('CancelRate:Q', format='.1f')]
).properties(
    width=CHART_WIDTH_BAR,
    height=CHART_HEIGHT_BUBBLE
)

# Footers
footer_text_avg = alt.Chart(df_dummy).mark_text(
    fontSize=13,fontWeight='normal', color='#666', dy=-15, text='Total Grants (Yearly Avg)'
).transform_filter(f"year_select == '{YEAR_ALL_INDICATOR}'")

footer_text_norm = alt.Chart(df_dummy).mark_text(
    fontSize=13,fontWeight='normal', color='#666', dy=-15, text='Total Grants'
).transform_filter(f"year_select != '{YEAR_ALL_INDICATOR}'")

footer_vol = alt.layer(
    footer_text_avg,
    footer_text_norm
).properties(
    width=CHART_WIDTH_BAR,
    height=FOOTER_HEIGHT
)

footer_impact = alt.Chart(df_dummy).mark_text(opacity=0).properties(
    width=CHART_WIDTH_BAR,
    height=FOOTER_HEIGHT
)

footer_center = alt.Chart(df_dummy).mark_text(opacity=0).properties(
    width=CHART_WIDTH_LABEL,
    height=FOOTER_HEIGHT
)

# Assembly
q2_right_part = alt.hconcat(
    alt.vconcat(
        header_vol_layer,
        alt.layer(def_left, drill_left),
        footer_vol
    ).resolve_scale(x='shared'),
    alt.vconcat(
        header_spacer,
        alt.layer(def_mid, drill_mid),
        footer_center
    ),
    alt.vconcat(
        header_risk_layer,
        alt.layer(def_right, drill_right),
        footer_impact
    ),
    spacing=10
).resolve_scale(y='independent')

q2_chart = alt.hconcat(
    q2_left_part,
    q2_right_part,
    spacing=HCONCAT_SPACING
).resolve_scale(color='independent').properties(
    title=alt.TitleParams(
        text='Grant Distribution and Cancellation Rate by Directorate',
        subtitle='Click a directorate bubble to drill-down into divisions. Only major directorates (≥100 total grants across 2021-2025)',
        fontSize=16, offset=-10
    )
)

In [25]:
# Q4 & Q5.1 LINE CHARTS (FINAL VERSION)

# Q4
q4_y_max = combined_q4['TotalBudget'].max() * 1.05
q4_color_scale = alt.Scale(domain=['All', 'Democrat', 'Republican'], range=[COLOR_ALL_PARTY, COLOR_DEMOCRAT, COLOR_REPUBLICAN])

q4_base = alt.Chart(combined_q4).transform_filter(
    "(party_filter == 'All' && datum.Group == 'All') || (datum.Group == party_filter)"
).transform_calculate(TotalBudgetBillions='datum.TotalBudget / 1000000000')

q4_line = q4_base.mark_line(strokeWidth=2).encode(
    x=alt.X('Year:O', axis=alt.Axis(labelAngle=0, title='Fiscal Year')),
    y=alt.Y('TotalBudgetBillions:Q', title='Budget ($B)', scale=alt.Scale(domain=[0, q4_y_max // 1000000000])),
    color=alt.Color('Group:N', scale=q4_color_scale, legend=alt.Legend(title='Party', orient='right')),
    opacity=alt.value(OPACITY_LINE)
)

q4_points = q4_base.mark_circle(size=POINT_SIZE_DEFAULT).encode(
    x='Year:O', y='TotalBudgetBillions:Q',
    color=alt.Color('Group:N', scale=q4_color_scale, legend=None),
    tooltip=['Year:O', 'Group:N', alt.Tooltip('TotalBudget:Q', format='$,.0f')]
)

q4_year_rule = alt.Chart(pd.DataFrame({'Year': YEARS_LIST})).mark_rule(
    strokeDash=[5, 5], strokeWidth=2, color='gray', opacity=OPACITY_YEAR_RULE
).encode(x='Year:O').transform_filter(f"year_select != '{YEAR_ALL_INDICATOR}'").transform_filter("datum.Year == year_select")

q4_chart = (q4_line + q4_points + q4_year_rule).properties(
    title='Grants Total Budget Evolution Over Time',
    width=LINE_CHART_WIDTH, height=BAR_HEIGHT - Q4_HEIGHT_OFFSET
)

# Q5.1
q5_all_avg = df_state_grants.groupby('Year').agg(GrantCount=('GrantCount', 'mean')).reset_index()
q5_all_avg['Group'] = 'All'
q5_all_avg['StateName'] = 'All States (Avg)'

q5_by_state = df_state_grants.copy()
q5_by_state['Group'] = q5_by_state['Party']

combined_q5_with_states = pd.concat([
    q5_all_avg[['Year', 'Group', 'GrantCount', 'StateName']],
    q5_by_state[['Year', 'Group', 'GrantCount', 'StateName', 'StateCode', 'Party']]
], ignore_index=True)

def create_segments(df):
    segments = []
    for state in df['StateName'].unique():
        state_df = df[df['StateName'] == state].sort_values('Year')
        years = state_df['Year'].values
        counts = state_df['GrantCount'].values
        groups = state_df['Group'].values
        for i in range(len(years) - 1):
            segments.append({
                'StateName': state, 'Year_from': years[i], 'Year_to': years[i + 1],
                'Count_from': counts[i], 'Count_to': counts[i + 1], 'Group': groups[i + 1]
            })
    return pd.DataFrame(segments)

q5_segments = create_segments(combined_q5_with_states)
q5_y_max = combined_q5_with_states['GrantCount'].max() * 1.05
q5_color_scale_grouped = alt.Scale(domain=['All', 'Democrat', 'Republican'], range=[COLOR_ALL_PARTY, COLOR_DEMOCRAT, COLOR_REPUBLICAN])

q5_line_base = alt.Chart(q5_segments).transform_filter(
    "!length(data('state_click_store')) ? datum.Group == 'All' : true"
).transform_filter(state_selection).transform_filter(
    "(party_filter == 'All') || (datum.Group == 'All') || (datum.Group == party_filter)"
)

q5_part1_line = q5_line_base.mark_rule(strokeWidth=2, opacity=OPACITY_LINE).encode(
    x=alt.X('Year_from:O', title='Fiscal Year', axis=alt.Axis(labelAngle=0)), x2='Year_to:O',
    y=alt.Y('Count_from:Q', title='Grant Count (Avg/Actual)', scale=alt.Scale(domain=[0, q5_y_max])), y2='Count_to:Q',
    color=alt.Color('Group:N', scale=q5_color_scale_grouped, legend=None)
)

q5_points_base = alt.Chart(combined_q5_with_states).transform_filter(
    "!length(data('state_click_store')) ? datum.Group == 'All' : true"
).transform_filter(state_selection).transform_filter(
    "(party_filter == 'All') || (datum.Group == 'All') || (datum.Group == party_filter)"
)

q5_part1_points = q5_points_base.mark_circle(size=POINT_SIZE_DEFAULT).encode(
    x='Year:O', y=alt.Y('mean(GrantCount):Q', scale=alt.Scale(domain=[0, q5_y_max])),
    color=alt.Color('Group:N', scale=q5_color_scale_grouped),
    tooltip=[alt.Tooltip('Year:O', title='Year'), alt.Tooltip('StateName:N', aggregate='min', title='State'),
             alt.Tooltip('Group:N', title='Category'), alt.Tooltip('mean(GrantCount):Q', title='Count', format='.0f')]
)

q5_year_rule = alt.Chart(pd.DataFrame({'Year': YEARS_LIST})).mark_rule(
    strokeDash=[5, 5], strokeWidth=2, color='gray', opacity=OPACITY_YEAR_RULE
).encode(x='Year:O').transform_filter(f"year_select != '{YEAR_ALL_INDICATOR}'").transform_filter("datum.Year == year_select")

q5_part1_chart = (q5_part1_line + q5_part1_points + q5_year_rule).properties(
    title={'text': 'State Grants Evolution (Average/Selected)'},
    width=LINE_CHART_WIDTH, height=LINE_CHART_HEIGHT
)

In [ ]:
# FINAL DASHBOARD ASSEMBLY

left_side = alt.vconcat(q1_row, q5_2_chart, spacing=VCONCAT_SPACING).resolve_scale(color='independent')

final_dashboard = left_side.add_params(
    year_param, topn_param, party_param
).properties(
    title=alt.TitleParams(
        text='NSF Grant Dashboard (2021-2025)',
        subtitle='Overview of grants and 2025 cancellations. Filter by Year/Party to explore',
        anchor='middle',
        fontSize=DASHBOARD_TITLE_FONT_SIZE,
        subtitleFontSize=DASHBOARD_SUBTITLE_FONT_SIZE,
        offset=DASHBOARD_TITLE_OFFSET
    )
).configure_view(stroke=None)

final_dashboard

alt.VConcatChart(...)